# ADIOS2-Python-Full-API-MPI: Write Read Global Values and Arrays

## Writer: Generate Global Values and Arrays

In [5]:
from mpi4py import MPI
import numpy as np
import adios2


# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Initialize dimensions
nx = 10
nsteps = 3

# Global array dimensions scattered across MPI nprocs
shape = [size * nx]
start = [rank * nx]
count = [   1 * nx]


# initialize ADIOS2
adios = adios2.ADIOS(comm)
io = adios.DeclareIO("VariableShapes")

# dummy np arrays to inform adios2 of the type
# Global values
var_nsteps = io.DefineVariable("nsteps", np.array([0], dtype=np.uint32))
var_size = io.DefineVariable("size", np.array([0], dtype=np.uint64))
var_step = io.DefineVariable("step") # string type
# Local values
var_rank = io.DefineVariable("rank", np.array([0], dtype=np.uint32), [adios2.LocalValueDim])
# Global array
# global array, there is a total shape, 
# and a local "window" for each rank (start and count)
var_global_array = io.DefineVariable("GlobalArray", np.array([0], dtype=np.float64), shape, start, count)
# Local array
# local array, no shape, no start offset
# only a local "dimension" for each rank (count)
var_local_array = io.DefineVariable("LocalArray", np.array([0], dtype=np.float32), [], [], count)

io.DefineAttribute("Greeting", "Hello ERDC variables from ADIOS2")

fw = io.Open("variable_shapes.bp", adios2.Mode.Write)

for step in range(0,3):
    
    fw.BeginStep()
    
    #global variable that doesn't change
    #save nsteps as a numpy array
    if(step == 0):
        fw.Put(var_nsteps, np.array(nsteps, dtype=np.uint32) )
        fw.Put(var_size, np.array(size, dtype=np.uint64))
    
    #global variable that changes over time
    #save step as a string
    fw.Put(var_step, str(step))
    
    # local values are independent quantities must be explicit
    # we can search for a variable name inside IO
    fw.Put(var_rank, np.array(rank, dtype=np.uint32) )
    fw.Put(var_global_array, np.arange(step, nx+step, dtype=np.float64))
    fw.Put(var_local_array, np.arange(step, nx+step, dtype=np.float32))
    
    fw.EndStep()
    

# CLose is always a good practice
fw.Close()

In [6]:
!bpls variable_shapes.bp -lavd -n 10

File info:
  of variables:  6
  of attributes: 1
  statistics:    Min / Max 

  double                  GlobalArray  3*{10} = 0 / 11
    (0,0)    0 1 2 3 4 5 6 7 8 9
    (1,0)    1 2 3 4 5 6 7 8 9 10
    (2,0)    2 3 4 5 6 7 8 9 10 11

  string                  Greeting     attr   = "Hello ERDC variables from ADIOS2"
  float                   LocalArray   3*[1]*{10} = 0 / 11
        step 0: 
          block 0: [0:9] = 0 / 9
    (0)    0 1 2 3 4 5 6 7 8 9
        step 1: 
          block 0: [0:9] = 1 / 10
    (0)    1 2 3 4 5 6 7 8 9 10
        step 2: 
          block 0: [0:9] = 2 / 11
    (0)    2 3 4 5 6 7 8 9 10 11

  unsigned int            nsteps       scalar = 3
3 

  unsigned int            rank         3*{1} = 0 / 0
    (0,0)    0 0 0 

  unsigned long long int  size         scalar = 1
1 

  string                  step         3*scalar = "" / ""
    (0)    "0" "1" "2" 

